# Multiple Activity Analyzer (MAYA)

Example:

```
chemical_multiverse(file='/content/example.csv', smiles_column_name='SMILES', target_activities=['Target_1', 'Target_2', 'Target_3'], MACCS=Falce, ECFP=True, MD=Falce, vPCA=True, t-SNE=True )
```



# Modules


In [ ]:
from IPython.utils import io
import tqdm.notebook
import os, sys, random
total = 100
with tqdm.notebook.tqdm(total=total) as pbar:
    with io.capture_output() as captured:
      # Instalar rdkit
      !pip -q install rdkit
      pbar.update(5)
      !pip install pandas
      !pip install dash
      pbar.update(10)
      # Instalar Numpy
      !pip install numpy
      # Instalar Pillow
      !pip -q install Pillow
      pbar.update(20)
      # Instalar Molvs
      !pip install -q molvs
      !pip install kora py3Dmol
      pbar.update(30)
      # Instalar Components
      !pip install dash-bootstrap-components
      pbar.update(50)
      # Instalar molplotly
      !pip install molplotly
      !pip install matplotlib
      pbar.update(60)
      # Chembl
      !pip install chembl_webresource_client
      pbar.update(70)
      #Seaborn
      !pip install seaborn
      # Instalar Pillow
      !pip -q install Pillow
      # Instalar jupyter-dash
      !pip install jupyter-dash
      pbar.update(80)
      # Instalar el diseño de aplicación dash
      !pip install dash-bootstrap-components
      !pip install pyngrok
      !pip install dash pyngrok
      pbar.update(100)

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
#Dataframe
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
! pip install kaleido
import kaleido
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
!pip install -U Flask
import dash
from dash import Dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pyngrok
from pyngrok import ngrok
import plotly.express as px
from dash import dcc, html
import plotly.express as px
import pandas as pd
import molplotly
import os
#Chemoinformatic
from rdkit import Chem, RDConfig
import csv
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.SaltRemover import SaltRemover
from rdkit import DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools, Scaffolds
from rdkit.Chem import Descriptors, Descriptors3D
import kora.install.rdkit
import py3Dmol
PandasTools.RenderImagesInAllDataFrames(images=True)
#Data frames
from rdkit.Chem import rdMolDescriptors
from molvs.standardize import Standardizer
from molvs.charge import Uncharger, Reionizer
from molvs.fragment import LargestFragmentChooser
from molvs.tautomer import TautomerCanonicalizer
from rdkit.Chem.rdmolops import GetFormalCharge, RemoveStereochemistry
from sklearn.preprocessing import StandardScaler
import dash_bootstrap_components as dbc
from sys import argv
from rdkit.Chem import MACCSkeys, RDKFingerprint, Pharm2D
from rdkit.Chem.Pharm2D import Gobbi_Pharm2D, Generate
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from rdkit.Chem.Scaffolds.MurckoScaffold import GetScaffoldForMol
from rdkit.Chem.Draw import IPythonConsole
import math
import plotly.express as px
import molplotly
from math import pi
from tempfile import TemporaryDirectory
import matplotlib.pyplot as plt
from IPython.display import display, SVG, HTML
import json
from pathlib import Path
!pip install kora py3Dmol
import kora.install.rdkit
import py3Dmol
from scipy.spatial import distance
import sklearn
!pip install torch
import torch
!pip install jupyter-dash
from jupyter_dash import JupyterDash
from google.colab.output import serve_kernel_port_as_iframe
import psutil
import subprocess
import time

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 11.0 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.0.6
    Uninstalling Werkzeug-3.0.6:
      Successfully uninstalled Werkzeug-3.0.6
  Attempting uninstall: Flask
    Found existing installation: Flask 3.0.3
    Uninstalling Flask-3.0.3:
      Successfully uninstalled Flask-3.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dash 2.18.2 requires Flask<3.1,>=1.0.4, but you have flask 3.1.0 which is incompatible.
dash 2.18.2 requires Werkzeug<3.1, but you have werkzeug 3.1.3 which is incompatible.


<ipython-input-2-c7d44d946328>:14: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-2-c7d44d946328>:15: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
Failed to find the pandas get_adjustment() function to patch
Failed to patch pandas - PandasTools will have limited functionality
Failed to patch pandas - unable to change molecule rendering


  Using cached werkzeug-3.0.6-py3-none-any.whl.metadata (3.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.6 MB/s eta 0:00:00
Using cached werkzeug-3.0.6-py3-none-any.whl (227 kB)
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


# Funtion

In [62]:
def chemical_multiverse(dataset=None, ID=None, smiles_column_name=None, target_activities=[None], vPCA=True, t_SNE=True, MACCS=True, ECFP=True, radius=3, druglikeness_descriptors=True, signaturizer_code=['B1','B4', 'C1', 'C2', 'E4', 'E5'], palette='RdBu_r', perplexity=33, size_point=12.0, size_point_representation='normal_desviation', evaluation_metric='mpIC50_value', metric_name='mpIC50',  point_shape='circle'):

   """
   Function designed to visualize a chemical multiverse with multiple activities or properties in an interactive scatter plot for a compound set.

   Args:
      dataset (CSV, XLSX, TSV, JSON, XML): DataFrame of compounds provided by the user.

      ID (str): Column name containing the ID of each compound.

      smiles_column_name (str): Column name containing SMILES notation.

      target_activities (list): List with the names of columns containing target activity values.

      vPCA (bool): User-defined variable for applying PCA.

      t_SNE (bool): User-defined variable for applying t-SNE.

      MACCS (bool): User-defined variable for applying MACCS keys.

      ECFP (bool): User-defined variable for applying ECFP.

      radius (int): User-defined variable for ECFP radius 2 or 3.

      druglikeness_descriptors (bool): User-defined variable for applying the 6 drug-likeness descriptors.

      signaturizer_code (list): User-defined variable for applying signaturizer with 6 different codes.

      palette (str): User-defined variable for defining a continuous color palette.

      perplexity (int): User-defined variable for adjusting the proximity of points in t-SNE, depending on dataset size.

      size_point (float): User-defined variable for adjusting point size in the plot.

      size_point_representation (str): User-defined variable for adjusting point size in the plot.

      point_shape (str): User-defined variable for adjusting point shape in the plot.

   Returns:
    Up to 18 interactive plots.
    DataFrame with all calculations performed.
   """




   def pretreatment(smi):

        """
        SMILES Preprocessing: from the SMILES column, filters by allowed atoms and applies the STD, LFC, UC, RI and TC functions.

        Args:
            smi (str): SMILES to preprocess; name of the column containing the SMILES.

        Returns:
            str: Preprocessed SMILES or the corresponding error message.
        """
        try:
            mol = Chem.MolFromSmiles(smi)
            if mol is None:
                return "Error 1"
            mol = STD(mol)
            mol = LFC(mol)
            allowed_elements = {"H","B","C","N","O","F","Si","P","S","Cl","Se","Br","I"}
            actual_elements = set([atom.GetSymbol() for atom in mol.GetAtoms()])
            if len(actual_elements-allowed_elements) == 0:
                mol = UC(mol)
                mol = RI(mol)
                mol = TC(mol)
                return Chem.MolToSmiles(mol)
            else:
                return "Error 2"
        except Exception as e:
            return f'Something else was found'


   def similarity_calc(smi1, smi2, method='tanimoto', fp_type='MACCS'):

        """
        Similarity calculation between two compounds based on their processed SMILES.

        Args:
            smi1 (str): SMILES of the first compound to compare.
            smi2 (str): SMILES of the second compound to compare.
            method (str): Comparison method to use. Can be 'tanimoto'
            fp_type (str): Type of fingerprint to use. Can be 'MACCS' or 'ECFP' with radius 2 or 3.

        Returns:
            similarity (float): Similarity coefficient between the compounds.
        """

        mol1 = Chem.MolFromSmiles(smi1)
        mol2 = Chem.MolFromSmiles(smi2)

        if fp_type == 'MACCS':
            fp1 = MACCSkeys.GenMACCSKeys(mol1)
            fp2 = MACCSkeys.GenMACCSKeys(mol2)
        elif fp_type == 'ECFP':
            fp1 = AllChem.GetMorganFingerprintAsBitVect(mol1, radius, nBits=2048)
            fp2 = AllChem.GetMorganFingerprintAsBitVect(mol2, radius, nBits=2048)

        if method == 'tanimoto':
            similarity = round(DataStructs.TanimotoSimilarity(fp1, fp2), 3)
        return similarity


   def evaluate_metric(data_frame, evaluation_metric='norma_desviation', metric_name='mpIC50'):
        """
        Generate a custom metric based on columns from a DataFrame and stores the result in a new column

        Arg:
           data_frame (str): pd.DataFrame, name of the dataframe containing the columns requerid for the evaluation
           evaluation_metric (str): Mathematical spression defining how to combine the columns. Example: '(-log(Activity1) + Activity2 - 0.5 * Activity3)'
           metric_name (str): Name of the new column where the results will be stored and the names showed in th graphics

        Returns:
           data_frame (pd.DataFrame): DataFrame with the new column
        """

        import re
        columns_in_expression = re.findall(r'[A-Za-z_][A-Za-z_0-9]*',evaluation_metric)

        missing_columns = [col for col in columns_in_expression if col not in data_frame.columns and not col.isnumeric()]
        if missing_columns:
          raise ValueError(f"Error: The following columns are missing in the provided dataset: {', '.join(missing_columns)}")


        expression_to_evaluate = evaluation_metric
        for col in columns_in_expression:
           if col in data_frame.columns:
             expression_to_evaluate = expression_to_evaluate.replace(col, f'data_frame["{col}"]')

        try:
          data_frame[metric_name] = eval(expression_to_evaluate)
        except Exception as e:
          raise ValueError(f"Error: {e}")

        return data_frame


   def configure_plot_layout(fig, data_frame):

      """
      Plot layout configuration.

      Args:
          fig: plot to configure.
          data_frame: DataFrame containing the plot data.

      Returns:
          fig: Plot with configured layout.
      """

      fig.update_layout(
          title_font=dict(size=25),
          plot_bgcolor='white',
          paper_bgcolor='white',
          xaxis=dict(mirror=True, showgrid=False, showline=True, zeroline=False, linecolor='black', title_font=dict(size=25),tickfont=dict(size=20)),
          yaxis=dict(mirror=True, showgrid=False, showline=True, zeroline=False, linecolor='black', title_font=dict(size=25), tickfont=dict(size=20)),
          coloraxis_colorbar=dict(title=metric_name, title_side= 'bottom', orientation="h", xanchor='center', yanchor='bottom', thickness=15, len=0.6, y=-0.29, title_font=dict(size=18), tickfont=dict(size=18)))
      plt.rcParams['font.family'] = 'Times New Roman'
      plt.figure(dpi=600)
      fig.update_traces(marker=dict(size=input_df[size_point_representation], symbol=point_shape, line=dict( width=2, color='black'), opacity=1))
      return fig


   def signaturizer_calculation(smiles_list, signaturizer_code):

        """
        Calculation of the chemical signature from a list of SMILES.

        Args:
            smiles_list (df): List of SMILES to process.
            signaturizer_code (list): Selected chemical signature code for calculation.

        Returns:
            signature_df: DataFrame containing the calculated chemical signature.
        """


        if signaturizer_code == 'B1': # Mechanismsof action
            signaturizer = Signaturizer(CCSpace.B1)
        if signaturizer_code == 'B4': # Binding
            signaturizer = Signaturizer(CCSpace.B4)
        if signaturizer_code == 'C1': # Small molecule roles
            signaturizer = Signaturizer(CCSpace.C1)
        if signaturizer_code == 'C2': # Small molecule pathways
            signaturizer = Signaturizer(CCSpace.C2)
        if signaturizer_code == 'E4': # Diseases & Toxicology
            signaturizer = Signaturizer(CCSpace.E4)
        if signaturizer_code == 'E5': # Drug-Drug interaction
            signaturizer = Signaturizer(CCSpace.E5)

        signature = signaturizer.infer_from_smiles(smiles_list)
        signature_df = pd.DataFrame(signature)

        return signature_df



   # Verification dataset is None
   if dataset is None:
      raise ValueError("The file name must be specified.")

   # Verification of the input dataset format
   file_readers = {

    'csv': pd.read_csv,
    'xlsx': pd.read_excel,
    'tsv': lambda f: pd.read_csv(f, delimiter='\t'),
    'json': pd.read_json,
    'xml': pd.read_xml
    }

   ext = dataset.split('.')[-1]

   # Read the dataset
   input_df = file_readers[ext](dataset)

   # Validation of required columns in the Dataframe
   if ext not in file_readers:
      raise ValueError("Unsupported file format")

   if ID not in input_df.columns:
    raise ValueError(f"Error: The column {ID}, is missing in the provided  dataset")

   if smiles_column_name not in input_df.columns:
      raise ValueError(f"Error: The column {smiles_column_name}, is missing in the provided  dataset")

   if any(target_activity not in input_df.columns for target_activity in target_activities):
      raise ValueError(f"Error: The following target activities are missing in the provided dataset: {', '.join(set(target_activities) - set(dataset.columns))}")


   # Validate types of variables
   if not isinstance(vPCA, bool):
      raise ValueError("Error: vPCA should be a boolean value (True or False)")

   if not isinstance(t_SNE, bool):
      raise ValueError("Error: t-SNE should be a boolean value (True or False)")

   if not isinstance(MACCS, bool):
      raise ValueError("Error: MACCS should be a boolean value (True or False)")

   if not isinstance(ECFP, bool):
      raise ValueError("Error: ECFP should be a boolean value (True or False)")

   if not isinstance(radius, int) or radius not in [2, 3]:
      raise ValueError("Error: radius should be an integer value of 2 or 3")

   if not isinstance(druglikeness_descriptors, bool):
      raise ValueError("Error: druglikeness_descriptors should be a boolean value (True or False)")

   if not isinstance(perplexity, int) or perplexity <= 0:
      raise ValueError("Error: perplexity should be an positive integer value")

   valid_signaturizer_codes = ['B1', 'B4', 'C1', 'C2', 'E4', 'E5']

   if not all(signaturizer_code in valid_signaturizer_codes for signaturizer_code in signaturizer_code):
    invalid_codes = [code for code in signaturizer_code if code not in valid_signaturizer_codes]
    raise ValueError(f"Error: Invalid signaturizer code(s): {', '.join(invalid_codes)}")

   if not isinstance(size_point, float) or size_point <= 0:
      raise ValueError("Error: size_point should be a positive float value")

   if not isinstance(point_shape, str):
      raise ValueError("Error: point_shape should be a string value")

   # It always necessary at least one dimensionality reduction technique

   if not MACCS and not ECFP:
       raise Warning('Warning: Both MACCS and ECFP are disable. You May want to enable at least one for similarity calculations')

   # Check for null values in key columns
   if input_df[smiles_column_name].isnull().any():
    raise ValueError("Error: The SMILES column contains null values. Please clean the data")

   if input_df[ID].isnull().any():
    raise ValueError("Error: The ID column contains null values. Pleas clean the data")

   # Check that ID is unique
   if not input_df[ID].is_unique:
    raise ValueError("Error: The ID column contains duplicate values. Please clean the data")

   # Validate target activities: check if values are numeric and non-null
   for activity in target_activities:
    if input_df[activity].isnull().any():
      raise ValueError(f'Error: The {activity} column contains null values. Please clean the data')
    if not pd.api.types.is_numeric_dtype(input_df[activity]):
      raise TypeError(f'Error: The {activity} column contains non-numeric values. Please ensure all values are numeric')


   # Definition of functions to processing SMILES
   STD = Standardizer()
   LFC = LargestFragmentChooser()
   UC = Uncharger()
   RI = Reionizer()
   TC = TautomerCanonicalizer()


   # Prossesing of Smiles and evaluation of activities
   input_df["Canonical_Smiles"] = input_df[smiles_column_name].apply(pretreatment)
   input_df = input_df[~input_df["Canonical_Smiles"].isin(["Error 1", "Error 2", "Error general"])].reset_index(drop=True)
   for activity_target in target_activities:
      input_df['mpIC50_value'] = -np.log(input_df[target_activities]).mean(axis=1)
      max_value = input_df['mpIC50_value'].max()
      min_value = input_df['mpIC50_value'].min()
      input_df['std'] = input_df[target_activities].std(axis=1)
      input_df['norma'] = ((input_df['std'] - input_df['std'].min()) / (input_df['std'].max() - input_df['std'].min()))
      input_df['normal_desviation'] = input_df['norma'] * (size_point-10)+9
      input_df['normal_desviation'] = input_df['normal_desviation'].fillna(10)


   # Calcualtion of descriptors
   smiles_list = input_df[[ID,"Canonical_Smiles"]]
   descriptors_df = pd.DataFrame({
            "HBA": [Descriptors.NumHAcceptors(y) for y in (Chem.MolFromSmiles(x) for x in smiles_list["Canonical_Smiles"])],
            "LogP": [Descriptors.MolLogP(y) for y in (Chem.MolFromSmiles(x) for x in smiles_list["Canonical_Smiles"])],
            "TPSA": [Descriptors.TPSA(y) for y in (Chem.MolFromSmiles(x) for x in smiles_list["Canonical_Smiles"])],
            "MW": [Descriptors.MolWt(y) for y in (Chem.MolFromSmiles(x) for x in smiles_list["Canonical_Smiles"])],
            "HBD": [Descriptors.NumHDonors(y) for y in (Chem.MolFromSmiles(x) for x in smiles_list["Canonical_Smiles"])],
            "RB": [Descriptors.NumRotatableBonds(y) for y in (Chem.MolFromSmiles(x) for x in smiles_list["Canonical_Smiles"])]
        }, index=smiles_list.index)
   descriptors_df['Canonical_Smiles'] = smiles_list['Canonical_Smiles']
   input_df = pd.merge(input_df, descriptors_df, on='Canonical_Smiles', how='inner')

   input_df['size_point_HBA'] = (7+np.log1p((descriptors_df['HBA'] - descriptors_df['HBA'].min()) / (descriptors_df['HBA'].max() - descriptors_df['HBA'].min()))*12)
   input_df['size_point_LogP'] = (7+np.log1p((descriptors_df['LogP'] - descriptors_df['LogP'].min()) / (descriptors_df['LogP'].max() - descriptors_df['LogP'].min()))*12)
   input_df['size_point_TPSA'] = (7+np.log1p((descriptors_df['TPSA'] - descriptors_df['TPSA'].min()) / (descriptors_df['TPSA'].max() - descriptors_df['TPSA'].min()))*12)
   input_df['size_point_MW'] = (7+np.log1p((descriptors_df['MW'] - descriptors_df['MW'].min()) / (descriptors_df['MW'].max() - descriptors_df['MW'].min()))*12)
   input_df['size_point_HBD'] = (7+np.log1p((descriptors_df['HBD'] - descriptors_df['HBD'].min()) / (descriptors_df['HBD'].max() - descriptors_df['HBD'].min()))*12)
   input_df['size_point_RB'] = (7+np.log1p((descriptors_df['RB'] - descriptors_df['RB'].min()) / (descriptors_df['RB'].max() - descriptors_df['RB'].min()))*12)

   input_df = evaluate_metric(input_df, evaluation_metric=evaluation_metric, metric_name=metric_name)

   # If MACCS keys are selected by the user, this section will build a matrix to calculate the similarity
   if MACCS is True:
              matrix_values=np.zeros((len(input_df), len(input_df)))

              for i in input_df.index:
                  for j in input_df.index:
                          a=input_df.loc[i].at["Canonical_Smiles"]
                          b=input_df.loc[j].at["Canonical_Smiles"]
                          alignment= similarity_calc(a,b, 'tanimoto', fp_type='MACCS')
                          matrix_values[i,j]=alignment
              matrix_MACCS = pd.DataFrame(matrix_values, index=input_df[ID], columns=input_df[ID])
              df_MACCS = pd.DataFrame(matrix_MACCS)


              # If user select PCA, this section will recover the values of similarity and train the model to calcualte principal componentes
              if vPCA is True:
                    data_MACCS = matrix_MACCS.iloc[:,0:].values
                    pd.DataFrame(data_MACCS)
                    label_MACCS = input_df.iloc[:,0].values
                    pca_MACCS = PCA(n_components=2)
                    pca_results_MACCS = pca_MACCS.fit_transform(data_MACCS)
                    label_MACCS = input_df[[ID, "Canonical_Smiles"]]
                    label_MACCS = label_MACCS.to_numpy()
                    arr = np.concatenate((label_MACCS,pca_results_MACCS), axis = 1)
                    data_MACCS_PCA = pd.DataFrame(data=arr, columns = [ID, "Canonical_Smiles", 'component1', 'component2'])
                    X_train_pca = pca_MACCS.fit_transform(data_MACCS)
                    exp_var_pca_maccs = pca_MACCS.explained_variance_ratio_
                    cum_sum_eigenvalues = np.cumsum(exp_var_pca_maccs)
                    loadings_maccs = pca_MACCS.components_.T * np.sqrt(pca_MACCS.explained_variance_[:2])
                    dataset_MACCS_PCA= pd.merge(data_MACCS_PCA, input_df, on=("Canonical_Smiles", ID), how='inner')
                    dataset_MACCS_PCA[['PC1 contribution', 'PC2 contribution']] = loadings_maccs
                    dataset_MACCS_PCA.to_csv('dataset_MACCS_PCA.csv')
                    x_label = f'component1 {exp_var_pca_maccs[0]}'
                    y_label = f'component2 {exp_var_pca_maccs[1]}'

                    # This section creates and customizes a scatter plot, generating an interactive plot and producing a downloadable file with higher resolution.
                    fig_pca = px.scatter(dataset_MACCS_PCA, x='component1', y='component2', color=metric_name, color_continuous_scale=palette, title='PCA - MACCS',
                      labels={'component1': f'Component 1 ({exp_var_pca_maccs[0]*100:.3}%)', 'component2': f'Component 2 ({exp_var_pca_maccs[1]*100:.3}%)'},
                      width=1200,
                      height=900,
                      #range_color=[min_value, max_value]
                                         )
                    fig_pca = configure_plot_layout(fig_pca, data_frame=dataset_MACCS_PCA)
                    app_maker_maccs_pca = molplotly.add_molecules(fig=fig_pca, df=dataset_MACCS_PCA, smiles_col="Canonical_Smiles", title_col=ID, color_col=metric_name, caption_cols=['Canonical_Smiles', 'PC1 contribution', 'PC2 contribution'])
                    serve_kernel_port_as_iframe("localhost")
                    app_maker_maccs_pca.run(port=8060)
                    #fig_pca.show()
                    pio.write_image(fig_pca, 'fig_PCA_MACCS_high_res.png', scale=10, width=1200, height=900)


              # "If the user sets the variable t_SNE to true, this section will retrieve similarity calculation values to train the model. It is necessary to define the perplexity value according to the dataset size
              if t_SNE is True:
                      data_tsne_MACCS = matrix_MACCS.copy()
                      tsne = TSNE(n_components=2, verbose=0, perplexity=perplexity, max_iter=500)
                      tsne_results_MACCS = tsne.fit_transform(data_tsne_MACCS)
                      label =input_df[[ID, "Canonical_Smiles"]]
                      label = label.to_numpy()
                      arr = np.concatenate((label, tsne_results_MACCS), axis = 1)
                      tsne_data = pd.DataFrame(data=arr, columns = [ID, "Canonical_Smiles", 'axis 1', 'axis 2'] )
                      all_dataset_tSNE = pd.merge(tsne_data, input_df, on=("Canonical_Smiles", ID), how='inner')
                      all_dataset_tSNE.to_csv('dataset_tSNE_MACCS.csv')

                      # Plot construction
                      fig_tsne_MACCS = px.scatter(all_dataset_tSNE, x='axis 1', y='axis 2', color=metric_name, color_continuous_scale=palette, title='T-SNE - MACCS',
                       labels={'axis 1': 'Axis 1', 'axis 2': 'Axis 2'},
                       width=1200,
                       height=900,
                       #range_color=[min_value, max_value]
                                                  )
                      fig_tsne_MACCS = configure_plot_layout(fig_tsne_MACCS, data_frame=all_dataset_tSNE)
                      app_maker_maccs_tsne = molplotly.add_molecules(fig=fig_tsne_MACCS, df=all_dataset_tSNE, smiles_col="Canonical_Smiles", title_col=ID, color_col=metric_name, caption_cols=['Canonical_Smiles'])
                      #fig_tsne_MACCS.show()
                      serve_kernel_port_as_iframe("localhost")
                      app_maker_maccs_tsne.run(port=8062)
                      pio.write_image(fig_tsne_MACCS, 'fig_tsne_MACCS_high_res.png', scale=10, width=1200, height=900)




  # If the user sets ECFP to true, this section will retrieve similarity_calc to generate a similarity matrix using the Extended Connectivity Fingerprint(ECFP). It is necessary to specify the radius to calculate either ECFP4 or ECFP6
   if ECFP is True:
                matrix_values_ECFP=np.zeros((len(input_df), len(input_df)))

                for i in input_df.index:
                    for j in input_df.index:
                          a=input_df.loc[i].at["Canonical_Smiles"]
                          b=input_df.loc[j].at["Canonical_Smiles"]
                          alignment= similarity_calc(a,b, 'tanimoto', fp_type='ECFP')
                          matrix_values_ECFP[i,j]=alignment
                          matrix_ECFP = pd.DataFrame(matrix_values_ECFP, index=input_df[ID], columns=input_df[ID])
                          df_ECFP = pd.DataFrame(matrix_ECFP)

                # This section performs PCA model training and calculates the principal components.
                if vPCA is True:
                      data_ECFP = matrix_ECFP.iloc[:,0:].values
                      pd.DataFrame(data_ECFP)
                      label_ECFP = input_df.iloc[:,0].values
                      pca_ECFP = PCA(n_components=2)
                      pca_results_ECFP = pca_ECFP.fit_transform(data_ECFP)
                      label_ECFP = input_df[[ID, "Canonical_Smiles"]]
                      label_ECFP = label_ECFP.to_numpy()
                      arr = np.concatenate((label_ECFP,pca_results_ECFP), axis = 1)
                      data_ECFP_PCA = pd.DataFrame(data=arr, columns = [ID, "Canonical_Smiles", 'component1', 'component2'])
                      sc = StandardScaler()
                      sc.fit(data_ECFP)
                      X_train_std = sc.transform(data_ECFP)
                      X_train_pca = pca_ECFP.fit_transform(X_train_std)
                      exp_var_pca_ecfp = pca_ECFP.explained_variance_ratio_
                      loadings_ecfp = pca_ECFP.components_.T * np.sqrt(pca_ECFP.explained_variance_[:2])
                      cum_sum_eigenvalues = np.cumsum(exp_var_pca_ecfp)
                      dataset_ECFP_PCA= pd.merge(data_ECFP_PCA, input_df, on=("Canonical_Smiles", ID), how='inner')
                      dataset_ECFP_PCA[['PC1 contribution', 'PC2 contribution']] = loadings_ecfp
                      dataset_ECFP_PCA.to_csv('dataset_ECFP_PCA.csv')
                      x_label = f'component1 {exp_var_pca_ecfp[0]}'
                      y_label = f'component2 {exp_var_pca_ecfp[1]}'

                      # Plot configuration
                      fig_pca_ECFP = px.scatter(dataset_ECFP_PCA, x='component1', y='component2', color=metric_name, color_continuous_scale=palette, title='PCA - ECFP',
                       labels={'component1': f'Component 1 ({exp_var_pca_ecfp[0]*100:.3}%)', 'component2': f'Component 2 ({exp_var_pca_ecfp[1]*100:.3}%)'},
                       width=1200,
                       height=900,
                       #range_color=[min_value, max_value]
                                                )
                      fig_pca_ECFP = configure_plot_layout(fig_pca_ECFP, data_frame=dataset_ECFP_PCA)
                      app_maker_ecfp_pca = molplotly.add_molecules(fig=fig_pca_ECFP, df=dataset_ECFP_PCA, smiles_col="Canonical_Smiles", title_col=ID, color_col=metric_name, caption_cols=['Canonical_Smiles', 'PC1 contribution', 'PC2 contribution'])
                      #fig_pca_ECFP.show()
                      serve_kernel_port_as_iframe("localhost")
                      app_maker_ecfp_pca.run(port=8061)
                      pio.write_image( fig_pca_ECFP, 'fig_PCA_ECFP_high_res.png', scale=10, width=1200, height=900)


                # t-SNE technique application with ECFP similarity calculation
                if t_SNE is True:
                      data_tsne_ECFP = matrix_ECFP.copy()
                      tsne = TSNE(n_components=2, verbose=0, perplexity=perplexity, max_iter=500)
                      tsne_results_ECFP = tsne.fit_transform(data_tsne_ECFP)
                      label =input_df[[ID, "Canonical_Smiles"]]
                      label = label.to_numpy()
                      arr = np.concatenate((label, tsne_results_ECFP), axis = 1)
                      tsne_data_ECFP = pd.DataFrame(data=arr, columns = [ID, "Canonical_Smiles", 'axis 1', 'axis 2'] )
                      dataset_tSNE_ECFP = pd.merge(tsne_data_ECFP, input_df, on=("Canonical_Smiles", ID), how='inner')
                      dataset_tSNE_ECFP.to_csv('dataset_tSNE_ECFP.csv')

                      # Plot configuration
                      fig_tsne_ECFP = px.scatter(dataset_tSNE_ECFP, x='axis 1', y='axis 2', color=metric_name, color_continuous_scale=palette, title='T-SNE - ECFP', labels={'axis 1': 'Axis 1', 'axis 2': 'Axis 2'},
                       width=1200,
                       height=900,
                       #range_color=[min_value, max_value]
                                                 )
                      fig_tsne_ECFP = configure_plot_layout(fig_tsne_ECFP, data_frame=dataset_tSNE_ECFP)
                      app_maker_ecfp_tsne = molplotly.add_molecules(fig=fig_tsne_ECFP, df=dataset_tSNE_ECFP, smiles_col="Canonical_Smiles", title_col=ID, color_col=metric_name, caption_cols=['Canonical_Smiles'])
                      #fig_tsne_ECFP.show()
                      serve_kernel_port_as_iframe("localhost")
                      app_maker_ecfp_tsne.run(port=8064)
                      pio.write_image( fig_tsne_ECFP, 'fig_tsne_ECFP_high_res.png', scale=10, width=1200, height=900)




   # This section calcualte Hidrogen Bond Aceptor(HBA), Hidrogen Bond Donator(HBD), Partitipon Coeficient(LogP), Total Polar Surface Area(TPSA), Molecular Weight(MW), Rotable Bonds(RB) descriptors form the processed SMILES
   if druglikeness_descriptors is True:
          smiles_list = pd.merge(smiles_list, descriptors_df, on='Canonical_Smiles', how='inner')
          descriptors_values = smiles_list.iloc[:,2:8].values
          molecular_descriptors_data_std = StandardScaler().fit_transform(descriptors_values) #Scaling of descriptor values
          df_MD = pd.DataFrame(molecular_descriptors_data_std)

          # Recover the DataFrame with scaled descriptor values to train the model and combine the ID, SMILES, and principal components into a single DataFrame.
          if vPCA is True:
              pca_druglikeness = PCA(n_components=2)
              pca_results_MD = pca_druglikeness.fit_transform(molecular_descriptors_data_std)
              label = input_df[[ID, "Canonical_Smiles"]]
              label = label.to_numpy()
              arr = np.concatenate((label,pca_results_MD), axis = 1)
              data_MD_PCA = pd.DataFrame(data=arr, columns = [ID, "Canonical_Smiles", 'component1', 'component2'])
              sc = StandardScaler()
              sc.fit(molecular_descriptors_data_std)
              X_train_std = sc.transform(molecular_descriptors_data_std)
              X_train_pca = pca_druglikeness.fit_transform(X_train_std)
              explained_variance_druglikeness = pca_druglikeness.explained_variance_ratio_
              loadings_druglikeness = pca_results_MD**2
              loadings_druglikeness = loadings_druglikeness / loadings_druglikeness.sum(axis=0)
              cum_sum_eigenvalues = np.cumsum(explained_variance_druglikeness)
              dataset_MD_PCA= pd.merge(data_MD_PCA, input_df, on=("Canonical_Smiles", ID), how='inner')
              dataset_MD_PCA[['PC1 contribution', 'PC2 contribution']] = loadings_druglikeness
              dataset_MD_PCA.to_csv('dataset_druglikeness_PCA.csv')
              x_label = f'component1 {explained_variance_druglikeness[0]}'
              y_label = f'component2 {explained_variance_druglikeness[1]}'

              # Plot configuration
              fig_pca_druglikeness = px.scatter(dataset_MD_PCA, x='component1', y='component2', color=metric_name, color_continuous_scale=palette, title='PCA - Drug-likeness descriptors', labels={'component1': f'Component 1 ({explained_variance_druglikeness[0]*100:.3}%)', 'component2': f'Component 2 ({explained_variance_druglikeness[1]*100:.3}%)'},
                 width=1200,
                 height=900,
                 #range_color=[min_value, max_value]
                                                )
              fig_pca_druglikeness = configure_plot_layout(fig_pca_druglikeness, data_frame=dataset_MD_PCA)
              app_maker = molplotly.add_molecules(fig= fig_pca_druglikeness, df=dataset_MD_PCA, smiles_col="Canonical_Smiles", title_col=ID, color_col=metric_name, caption_cols=['Canonical_Smiles', 'HBA', 'LogP', 'TPSA', 'MW', 'HBD', 'RB', 'PC1 contribution', 'PC2 contribution'])
              #fig_pca_MD.show()
              serve_kernel_port_as_iframe("localhost")
              app_maker.run(port=8066)
              pio.write_image( fig_pca_druglikeness, 'fig_pca_druglikeness_high_res.png', scale=10, width=1200, height=900)


          # run the t_SNE model with scaled descriptors values
          if t_SNE is True:
                tsne = TSNE(n_components=2, verbose=0, perplexity=perplexity, max_iter=500)
                tsne_results_MD = tsne.fit_transform(molecular_descriptors_data_std)
                label =input_df[[ID, "Canonical_Smiles"]]
                label = label.to_numpy()
                arr = np.concatenate((label, tsne_results_MD), axis = 1)
                tsne_data_MD = pd.DataFrame(data=arr, columns = [ID, "Canonical_Smiles", 'axis 1', 'axis 2'] )
                dataset_tSNE_MD = pd.merge(tsne_data_MD, input_df, on=("Canonical_Smiles", ID), how='inner')
                dataset_tSNE_MD.to_csv('dataset_druglikeness_tSNE.csv')

                # Plot configuration
                fig_tsne_druglikeness = px.scatter(dataset_tSNE_MD, x='axis 1', y='axis 2', color=metric_name, color_continuous_scale=palette, title='T-SNE - Drug-likeness descriptors', labels={'axis 1': 'Axis 1', 'axis 2': 'Axis 2'},
                  width=1200,
                  height=900,
                  #range_color=[min_value, max_value]
                                                   )
                fig_tsne_druglikeness = configure_plot_layout(fig_tsne_druglikeness, data_frame=dataset_tSNE_MD)
                app_maker = molplotly.add_molecules(fig=fig_tsne_druglikeness, df=dataset_tSNE_MD, smiles_col="Canonical_Smiles", title_col=ID, color_col=metric_name, caption_cols=['Canonical_Smiles'])
                #fig_tsne_MD.show()
                serve_kernel_port_as_iframe("localhost")
                app_maker.run(port=8068)
                pio.write_image(fig_tsne_druglikeness, 'fig_tsne_druglikeness_high_res.png', scale=10, width=1200, height=900)



   # If user decicde to obtein bioactive descriptors, this section will recovery the signaturizer_calcualtion funtion to build a dataframe with the values of the signaturizes selectioned
   if signaturizer_code is not None:
    !pip install signaturizer3d
    from signaturizer3d import Signaturizer, CCSpace

    for code in signaturizer_code:
      smiles = input_df[[ID,'Canonical_Smiles']]
      signature_df = signaturizer_calculation(smiles_list=smiles['Canonical_Smiles'], signaturizer_code = code)


      # With signaturizer fingerprint we train PCA model
      if vPCA is True:
              pca_signatures = PCA(n_components=2)
              pca_results_mechanisms_of_action = pca_signatures.fit_transform(signature_df)
              label = input_df[[ID, "Canonical_Smiles"]]
              label = label.to_numpy()
              arr = np.concatenate((label,pca_results_mechanisms_of_action), axis = 1)
              data_mechanism_PCA = pd.DataFrame(data=arr, columns = [ID, "Canonical_Smiles", 'component1', 'component2'])
              sc = StandardScaler()
              sc.fit(signature_df)
              X_train_std = sc.transform(signature_df)
              X_train_pca = pca_signatures.fit_transform(X_train_std)
              exp_var_pca_signatures = pca_signatures.explained_variance_ratio_
              cum_sum_eigenvalues = np.cumsum(exp_var_pca_signatures)
              loadings_signatures = pca_results_mechanisms_of_action**2
              loadings_signatures = loadings_signatures / loadings_signatures.sum(axis=0)
              dataset_mechanism_PCA= pd.merge(data_mechanism_PCA, input_df, on=("Canonical_Smiles", ID), how='inner')
              dataset_mechanism_PCA[['PC1 contribution', 'PC2 contribution']] = loadings_signatures
              dataset_mechanism_PCA.to_csv(f'dataset_{code}_PCA.csv')
              x_label = f'component1 {exp_var_pca_signatures[0]}'
              y_label = f'component2 {exp_var_pca_signatures[1]}'

              # Plot configuration
              fig_pca_mechanisim = px.scatter(dataset_mechanism_PCA, x='component1',y='component2', color=metric_name, color_continuous_scale=palette, title=(f'PCA - {code}'), labels={'component1': f'Component 1 ({exp_var_pca_signatures[0]*100:.3}%)', 'component2': f'Component 2 ({exp_var_pca_signatures[1]*100:.3}%)'},
               width=1200,
               height=900,
               #range_color=[min_value, max_value]
                                              )
              fig_pca_mechanisim = configure_plot_layout(fig_pca_mechanisim, data_frame=dataset_mechanism_PCA)
              app_maker = molplotly.add_molecules(fig=fig_pca_mechanisim, df=dataset_mechanism_PCA, smiles_col="Canonical_Smiles", title_col=ID, color_col=metric_name, caption_cols=['Canonical_Smiles', 'PC1 contribution', 'PC2 contribution'])
              #fig_pca_mechanisim.show()
              serve_kernel_port_as_iframe("localhost")
              app_maker.run(port=8070)
              pio.write_image(fig_pca_mechanisim, f'fig_pca_{code}_high_res.png', scale=10, width=1200, height=900)

      # Run t_SNE model to signaturizer fingerprint
      if t_SNE is True:
                tsne = TSNE(n_components=2, verbose=0, perplexity=perplexity, max_iter=500)
                tsne_results_signatures = tsne.fit_transform(signature_df)
                label =input_df[[ID, "Canonical_Smiles"]]
                label = label.to_numpy()
                arr = np.concatenate((label, tsne_results_signatures), axis = 1)
                tsne_data_signatures = pd.DataFrame(data=arr, columns = [ID, "Canonical_Smiles", 'axis 1', 'axis 2'] )
                dataset_tSNE_signatures = pd.merge(tsne_data_signatures, input_df, on=("Canonical_Smiles", ID), how='inner')
                dataset_tSNE_signatures.to_csv(f'dataset_{code}_tSNE.csv')

                # Plot config
                fig_tsne_signaturizer = px.scatter(dataset_tSNE_signatures, x='axis 1', y='axis 2', color=metric_name, color_continuous_scale=palette, title=(f't-SNE - {code}'), labels={'axis 1': 'Axis 1', 'axis 2': 'Axis 2'},
                    width=1200,
                    height=900,
                    #range_color=[min_value, max_value]
                                                   )
                fig_tsne_signaturizer = configure_plot_layout(fig_tsne_signaturizer, data_frame=dataset_tSNE_signatures)
                app_maker = molplotly.add_molecules(fig=fig_tsne_signaturizer, df=dataset_tSNE_signatures, smiles_col="Canonical_Smiles", title_col=ID, color_col=metric_name, caption_cols=['Canonical_Smiles'])
                #fig_tsne_signaturizer.show()
                serve_kernel_port_as_iframe("localhost")
                app_maker.run(port=8072)
                pio.write_image(fig_tsne_signaturizer, f'fig_tsne_{code}_high_res.png', scale=10, width=1200, height=900)

# Example

In [63]:
chemical_multiverse(dataset='/content/study_case_1.xlsx', smiles_column_name='Smiles', ID='molecule_chembl_id', target_activities=['IC50_Target_1', 'IC50_Target_2'], perplexity=15, size_point_representation='size_point_RB', evaluation_metric='IC50_Target_1 - IC50_Target_2', metric_name='Overall_Activity')

/usr/local/lib/python3.10/dist-packages/dash/dash.py:579: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

/usr/local/lib/python3.10/dist-packages/dash/dash.py:579: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



<IPython.core.display.Javascript object>

...

<Figure size 3840x2880 with 0 Axes>

<Figure size 3840x2880 with 0 Axes>

<Figure size 3840x2880 with 0 Axes>

<Figure size 3840x2880 with 0 Axes>

<Figure size 3840x2880 with 0 Axes>